In [1]:
# install in conda environment
# !conda install -c conda-forge pydub
# !conda install -c conda-forge ffmpeg

from os import path
from pydub import AudioSegment
import pandas as pd
import numpy as np
import pickle

import re

In [2]:
from os import path
from pydub import AudioSegment

src = "Roar_of_the_Wild.mp3"
dst = "test.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

FileNotFoundError: [Errno 2] No such file or directory: 'Roar_of_the_Wild.mp3'

In [3]:
import pandas as pd

df = pd.read_csv('../fma_metadata/tracks.csv', header=None)

# getting names for columns and index:
cnames = zip(df.iloc[0,1:], df.iloc[1,1:])
inames = list(df.iloc[0,:1])   

#drop the rows with column names (for columns and index)
df.drop([0,1],axis=0,inplace=True)
#set the indexes
df.set_index([0],inplace=True)
# set the names for columns and indexes
df.columns = pd.MultiIndex.from_tuples(cnames)
df.index.names = inames

df['set', 'subset'].head()
 
#df = df[(df['set', 'subset'] == 'small') | (df['set', 'subset'] == 'medium') | (df['set', 'subset'] == 'large')]

df.info()

/Users/estberg/anaconda2/envs/490-final-project/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
<class 'pandas.core.frame.DataFrame'>
Index: 106575 entries, track_id to 155320
Data columns (total 52 columns):
(album, comments)              106574 non-null object
(album, date_created)          103045 non-null object
(album, date_released)         70294 non-null object
(album, engineer)              15295 non-null object
(album, favorites)             106574 non-null object
(album, id)                    106574 non-null object
(album, information)           83149 non-null object
(album, listens)               106574 non-null object
(album, producer)              18060 non-null object
(album, tags)                  106574 non-null object
(album, title)          

In [4]:
# adds an is_classical column
df['track', 'is_classical'] = False
count = 0
for i in range(1, len(df)):
    res = df['track', 'genres_all'].iloc[i].strip('][').split(', ')
    if ('5' in res):
        df['track', 'is_classical'].iloc[i] = True
        count += 1

/Users/estberg/anaconda2/envs/490-final-project/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
def map_regex(dict, text):

    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())), flags=re.IGNORECASE)

    if regex.search(text):
        ret = regex.search(text)
        return dict[ret.group()]
    else:
        return None

pd.set_option('display.max_rows', 1000)
#classical = df[df['track', 'genre_top'] == 'Classical']
classical = df[df['track', 'is_classical'] == True]
print(len(classical))

info = pd.concat([classical['track', 'composer'], classical['track', 'title']], axis=1)
print("initial data length: ", len(info[info['track', 'composer'].notnull()]))

contain_composers = [r'.*Bach.*', r'.*Brahms.*', r'.*Mozart.*', r'.*Schubert.*', r'.*Beethoven.*', r'.*Paganini.*', r'.*Chopin.*', r'.*Grieg.*', r'.*Debussy.*', r'.*Mussorgsky.*', r'.*Liszt.*', r'.*Rachmaninoff.*', r'.*Schumann.*', r'.*Mendelssohn.*', r'.*Alkan.*', r'.*Vivaldi.*', r'.*Wagner.*', r'.*Satie.*', r'.*Camille.*', r'.*Pachelbel.*', r'.*Palestrina.*', r'.*Pizzetti.*', r'.*Bizet.*', r'.*Tchaikovsky.*', r'.*Dvorak.*', r'.*Handel.*', r'.*Bartok.*', r'.*Corelli.*', r'.*Albinoni.*', r'.*Ravel.*', r'.*Spohr.*', r'.*Cambini.*', r'.*Haydn.*', r'.*Faure.*', r'.*Schumann.*', r'.*Kirschner.*', r'.*Purcell.*', r'.*Sigismondo.*', r'.*Merula.*', r'.*Strozzi.*', r'.*Frescobaldi.*', r'.*Paganini.*', r'.*Gibbons.*', r'.*Byrd.*', r'.*Papa.*', r'.*Monteverdi.*', r'.*Lobo.*', r'.*Tallis.*', r'.*Rore.*', r'.*Stabile.*', r'.*Ferrabosco.*', r'.*Dvořák.*', r'.*Victoria.*', r'.*Animuccia.*', r'.*Ives.*', r'.*Janáček.*', r'.*Wolf.*', r'.*Danzi.*']
composers = ['Bach', 'Brahms', 'Mozart', 'Schubert', 'Beethoven', 'Paganini', 'Chopin', 'Grieg', 'Debussy', 'Mussorgsky', 'Liszt', 'Rachmaninoff', 'Schumann', 'Mendelssohn', 'Alkan', 'Vivaldi', 'Wagner', 'Satie', 'Camille', 'Pachelbel', 'Palestrina', 'Pizzetti', 'Bizet', 'Tchaikovsky', 'Dvorak', 'Handel', 'Bartok', 'Corelli', 'Albinoni', 'Ravel', 'Spohr', 'Cambini', 'Haydn', 'Faure', 'Schumann', 'Kirschner', 'Purcell', 'Sigismondo', 'Merula', 'Strozzi', 'Frescobaldi', 'Paganini', 'Gibbons', 'Byrd', 'Papa', 'Monteverdi', 'Lobo', 'Tallis', 'Rore', 'Stabile', 'Ferrabosco', 'Dvořák', 'Victoria', 'Animuccia', 'Ives', 'Janáček', 'Wolf', 'Danzi']
composer_to_period = {
    "Albinoni": "Bar",
    "Alkan": "Rom",
    "Animuccia": "Ren",
    "Bach": "Bar",
    "Bartok": "Mod",
    "Beethoven": "Cla",
    "Bizet": "Rom",
    "Brahms": "Rom",
    "Byrd": "Bar",
    "Cambini": "Cla",
    "Camille": "Rom", # Saint-Saens?
    "Chopin": "Rom",
    "Corelli": "Bar",
    "Danzi": "Cla",
    "Debussy": "Rom",
    "Dvorak": "Rom",
    "Dvořák": "Rom",
    "Faure": "Rom",
    "Ferrabosco": "Bar",
    "Frescobaldi": "Bar",
    "Gibbons": "Bar",
    "Grieg": "Rom",
    "Handel": "Cla",
    "Haydn": "Cla",
    "Ives": "Mod",
    "Janáček": "Rom",
    "Kirschner": "Mod", #Experimental Classical
    "Liszt": "Rom",
    "Lobo": "Bar",
    "Mendelssohn": "Rom",
    "merula": "Bar",
    "Monteverdi": "Bar",
    "Mozart": "Cla",
    "Mussorgsky": "Rom",
    "Pachelbel": "Cla",
    "Paganini": "Cla",
    "Paganini": "Cla",
    "Palestrina": "Ren",
    "Papa": "Ren",
    "Pizzetti": "Mod",
    "Purcell": "Bar",
    "Rachmaninoff": "Rom",
    "Ravel": "Mod",
    "Rore": "Ren",
    "Satie": "Rom",
    "Schubert": "Rom",
    "Schumann": "Rom",
    "Schumann": "Rom",
    "Sigismondo": "Bar", #Sigismondo D'India
    "Spohr": "Rom",
    "Stabile": "Ren",
    "Strozzi": "Bar",
    "Tallis": "Ren",
    "Tchaikovsky": "Rom",
    "Victoria": "Ren",
    "Vivaldi": "Cla",
    "Wagner": "Rom",
    "Wolf": "Rom"
}

info['track', 'comp_from_title'] = info['track', 'title'].replace(contain_composers, composers, regex=True)
info['track', 'comp_from_title'].loc[~info['track', 'comp_from_title'].isin(composers)] = np.NaN
info['track', 'composer'] = info['track', 'composer'].fillna(info['track', 'comp_from_title'])
info['track', 'period'] = info['track', 'composer'].apply(lambda v: map_regex(composer_to_period, str(v)))

print("final data length: ", len(info[info['track', 'composer'].notnull()]))
print("period length: ", len(info[info['track', 'period'].notnull()]))

print(info['track', 'composer'].unique())

print(info)
info.to_csv("track_info.csv")

file_path = 'track_info.pkl'
labels_file = open(file_path,'wb')
pickle.dump(info, labels_file)
labels_file.close()


4106
initial data length:  1298
final data length:  1595
period length:  794
[nan 'Robert Schumann' 'Georges Bizet' 'Ludwig van Beethoven'
 'Modest Mussorgsky' 'Brahms' 'Mozart' 'Schubert' 'Bach' 'Beethoven'
 'Grieg' 'No Regrets For Our Youth' 'Kenneth Kirschner' 'Henry Purcell'
 "Sigismondo D'India" 'Tarquinio merula' 'Barbara Strozzi'
 'Girolamo Frescobaldi' 'Igor Ballereau' 'Paganini' 'Chopin' 'Liszt'
 'Debussy' 'Mussorgsky' 'Ives' 'Dvořák' 'Rachmaninoff' 'Schumann' 'Bartok'
 'Albinoni' 'Satie' 'Michael Tilson Thomas' 'Claude Debussy' 'Mendelssohn'
 'Cooper-Moore' 'Charles-Valentin Alkan' 'James Beaudreau'
 'Orlando Gibbons' 'William Byrd' 'G P da Palestrina'
 'Jacobus Clemens non Papa' 'Claudio Monteverdi' 'Alonso Lobo'
 'Thomas Tallis' 'Cipriano de Rore' 'Annibale Stabile'
 'Alfonso Ferrabosco' 'Tomas Luis de Victoria' 'Giovanni Animuccia'
 ' G P da Palestrina' 'Palestrina' 'Pizzetti' 'Strauss' 'Camille'
 'Vivaldi' 'Igor Stravinsky' 'Bizet' 'Haydn' 'Maurice Ravel'
 'Wolfgang Amade

In [8]:
metadata_url = "https://homes.cs.washington.edu/~thickstn/media/musicnet_metadata.csv"

composer_to_period = {
    "Schubert": "Rom", 
    "Mozart": "Cla", 
    "Dvorak": "Rom",
    "Cambini": "Cla", 
    "Haydn": "Cla", 
    "Brahms": "Rom", 
    "Faure": "Rom", 
    "Ravel": "Mod", 
    "Bach": "Bar", 
    "Beethoven": "Cla"
    }

musicnet = pd.read_csv(metadata_url)
musicnet['period'] = musicnet['composer'].map(composer_to_period)

# create pkl file mapping file id to time period
file_path = 'musicnet_labels.pkl'
id_to_period = musicnet.set_index('id')['period'].to_dict()
labels_file = open(file_path,'wb')
pickle.dump(id_to_period, labels_file)
labels_file.close()

In [37]:
# read file
filename = "musicnet_labels.pkl"
with open(filename, 'rb') as handle:
    id2label = pickle.load(handle)
    
print(id2label)

{1727: 'Rom', 1728: 'Rom', 1729: 'Rom', 1730: 'Rom', 1733: 'Rom', 1734: 'Rom', 1735: 'Rom', 1739: 'Rom', 1742: 'Rom', 1749: 'Rom', 1750: 'Rom', 1751: 'Rom', 1752: 'Rom', 1755: 'Rom', 1756: 'Rom', 1757: 'Rom', 1758: 'Rom', 1759: 'Rom', 1760: 'Rom', 1763: 'Rom', 1764: 'Rom', 1765: 'Rom', 1766: 'Rom', 1768: 'Rom', 1771: 'Rom', 1772: 'Rom', 1773: 'Rom', 1775: 'Rom', 1776: 'Rom', 1777: 'Rom', 1788: 'Cla', 1789: 'Cla', 1790: 'Cla', 1791: 'Cla', 1792: 'Cla', 1793: 'Cla', 1805: 'Cla', 1807: 'Cla', 1811: 'Cla', 1812: 'Cla', 1813: 'Cla', 1817: 'Cla', 1818: 'Cla', 1819: 'Cla', 1822: 'Cla', 1824: 'Cla', 1828: 'Cla', 1829: 'Cla', 1835: 'Cla', 1859: 'Cla', 1872: 'Cla', 1873: 'Cla', 1876: 'Cla', 1893: 'Cla', 1916: 'Rom', 1918: 'Rom', 1919: 'Rom', 1922: 'Rom', 1923: 'Rom', 1931: 'Rom', 1932: 'Rom', 1933: 'Rom', 2075: 'Cla', 2076: 'Cla', 2077: 'Cla', 2078: 'Cla', 2079: 'Cla', 2080: 'Cla', 2081: 'Cla', 2082: 'Cla', 2083: 'Cla', 2104: 'Cla', 2105: 'Cla', 2106: 'Cla', 2112: 'Rom', 2113: 'Rom', 2114: 'Rom'